In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import collections
# read word2vec wiki-news-300d-1M.vec
#from gensim.models import KeyedVectors
#model = KeyedVectors.load_word2vec_format('/USERS/d8182103/facebook_data/embedding/wiki-news-300d-1M.vec')

In [14]:
with open('ti_du_xia_jiang_xun_lian_fa.md','r') as f:
    lines = f.read()

In [15]:
with open('ti_du_xia_jiang_xun_lian_fa.txt','w') as f:
    for e,l in enumerate(lines.split('$$')):
        if e%2!=0:
            f.write('`$'+l+'$`')
        else:
            f.write(l)

In [9]:
alltext

'# 为什么神经网络能够识别\n\n为了研究神经网络，我们必须要对什么网络是什么有一个更直观的认识。\n##基本变换：层\n***一、神经网络是由一层一层构建的，那么每层究竟在做什么？***\n\n - **数学式子**：`$\\vec{y}= a(W\\cdot\\vec{x} + {b})$`，其中`$\\vec{x}$`是输入向量，`$\\vec{y}$`是输出向量，`$\\vec{b}$`是偏移向量，`$W$`是权重矩阵，`$a()$`是激活函数。每一层仅仅是把输入`$\\vec{x}$`经过如此简单的操作得到`$\\vec{y}$`。\n - **数学理解**：通过如下5种对输入空间（输入向量的集合）的操作，完成![](transform.png)的变换(矩阵的行空间到列空间)。\n \n **注**：用“空间”二字是指被分类的并不是单个事物，而是**一类**事物。空间是指这类事物所有个体的集合。\n \n      - **1.** 升维/降维   \n      - **2.** 放大/缩小   \n      - **3.** 旋转  \n      - **4.** 平移   \n      - **5.** “弯曲”\n      \n      ![](/assets/1layer.gif)\n      \n      这5种操作中，1,2,3的操作由`$W\\cdot\\vec{x}$`完成，4的操作是由`$+ \\vec{b}$`完成，5的操作则是由`$a()$`来实现。\n      \n\n\n>每层的数学理解：**用线性变换跟随着非线性变化，将输入空间投向另一个空间**。\n\n\n - **物理理解**：对 `$W\\cdot\\vec{x}$` 的理解就是**通过组合形成新物质**。`$a()$`又符合了我们所处的世界都是非线性的特点。\n  - **假想情景：**`$\\vec{x}$`是二维向量，维度是碳原子和氧原子的数量 `$[C； O]$`，数值且定为`$[1； 1]$`。若确定`$\\vec{y}$`是三维向量，就会形成如下网络的形状 (神经网络的每个节点表示一个维度)。通过改变权重的值，可以获得若干个不同物质。右侧的节点数决定了想要获得多少种不同的新物质（矩阵的行数）。\n  \n    ![二维--->三维](ht

In [8]:
# filter text on char level
def textfilter(x):
    # replace [inaudible 00:+] to inaudible
    x = re.sub(r'\[[^:]*(\d\d:\d\d|\d\d:\d\d:\d\d)[\s\w\d]*\]',' inaudible ',x)
    # replace email
    x = re.sub(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+|e-mail)",' email ',x)
    
    # replace email
    x = re.sub(r'[a-zA-Z0-9-]+\.[a-z]{2,4}',' url ',x)
    # replace time
    x = re.sub(r'( \d\d:\d\d| \d\d| \d):\d\d( \w.m.|)',' a.m. ',x)
    # replace num
    x = re.sub(r' \d\.*\d*[ |,]\d*',' 7 ',x)
    x = re.sub(r' \d+[\/|\:]\d+',' 7 ',x)
    x = re.sub(r'Q&A[|s]',' question-and-answer ',x)
    x = re.sub(r'_',' ',x)
    x = re.sub(r"n\'t",' nt ',x)
    x = re.sub(r"[\`\`|\'\']",'"',x)
    x = re.sub(r"o\'clock",' a.m. ',x)
    x = re.sub(r'"',' ',x)
    x = re.sub(r'f\*\*\*','fuck',x)
    x = re.sub(r's\*\*\*','shit',x)
    return x.lower()

# make word to ID lookup table
make word embedding matrix from pretrained model based on lookup table

In [9]:
# filter all text in train/test/vali sets and tokenize them with start and end mark
# get all words 
PATH = '/USERS/d8182103/'
traindf = pd.read_csv(PATH+'firstimpressionV2/train/text_and_labels.csv')
testdf = pd.read_csv(PATH+'firstimpressionV2/test/text_and_labels.csv')
validf = pd.read_csv(PATH+'firstimpressionV2/vali/text_and_labels.csv')
text = pd.concat([traindf['text'],testdf['text'],validf['text']])
all_words = []
for i in range(len(text)):
    if text.iloc[i] is not np.NaN:
        re_text = textfilter(text.iloc[i])
        # delete half-words
        if '-' in re_text[-5:]:
            splits = re_text.split('-')
            re_text= '-'.join(splits[:-1])+ ' '+splits[-1]
        if '-' in re_text[:5]:
            splits = re_text.split('-')
            re_text=splits[0] + ' ' +'-'.join(splits[1:])
        #remove / and ' = from word
        tokens = [re.sub(r"[\/|\'|\=]",'',w) for w in word_tokenize(re_text)]
        tokens = ['text-start']+tokens+['text-end']
        all_words = all_words+tokens

In [61]:
V1traindf = pd.read_csv(PATH+'firstimpressionV1/train/text_and_labels.csv')
V2traindf = pd.read_csv(PATH+'firstimpressionV2/train/text_and_labels.csv')

In [62]:
np.save(PATH+'firstimpressionV1/tfrecord/big5/train/mean_std.npy',[np.array(V1traindf.mean(),dtype='float32'),np.array(V1traindf.std(),dtype='float32')])
np.save(PATH+'firstimpressionV2/tfrecord/big5/train/mean_std.npy',[np.array(V2traindf.mean(),dtype='float32'),np.array(V2traindf.std(),dtype='float32')])

In [10]:
# count words
def find_fre(all_words, n_times):
    frequencies = collections.Counter(all_words).most_common()
    index = 0
    for f in frequencies:
        if f[1] == n_times:
            return index
        else:
            index +=1
    unique_words=len(frequencies)
    return unique_words
print("#words appearing >= 1 time: %d" %find_fre(all_words, 0))
print("#words appearing >= 2 time: %d" %find_fre(all_words, 1))
print("#words appearing >= 3 time: %d" %find_fre(all_words, 2))
print("#words appearing >= 4 time: %d" %find_fre(all_words, 3))

#words appearing >= 1 time: 15069
#words appearing >= 2 time: 8163
#words appearing >= 3 time: 5822
#words appearing >= 4 time: 4598


In [11]:
# count words and check whether they are OOV
# delete words that are OOV
frequencies = collections.Counter(all_words)
keys = [key for key in frequencies]
for key in keys:
    try:
        model.most_similar(key)
    except KeyError:
        if key !='text-end' and key !='text-start':
            print(key)
            frequencies.pop(key)
len(frequencies)

j.c.
multiplayers
darkstalkers
100k-ish
brizzell
togashi
mari2
fanta-sci
micro-focus
macklemore
sheeran
morgans
mervin
elgato
hd60
never-world
usopp
suaves
arpock
arock
capitillo
noselipseptum
jethro
asmr
jannie
rga
charizard
charmander
patreon
anotonio
caden
phasma
twittie
twitties
dychesfamq
dychesfamqanda
shulman
yanni
rimmel
stimu
altoids
malloch
correctionals
amttg
koll
slgs
dykster
monitization
pre-shoot
jay-man
eriktv365
dipbrow
ab-work
graceffa
blaha
wheeder
bradwest
doncamatic
gorillaz
officialfirephoenix
leena
justkissmyfrog
beautycon
yiannopoulos
garaholic
csas
rkt
gmos
geo-cashing
bellingham
motherxteresa
ophelia
lovibond
leafgreen
iffyier
am-raps
auto-regulates
am-rap
mommyhood
berneseoftherockies
kayden
post-christmas
freesa
therese
eviegilson
vlogmas
manhasset
garnier
volumizer
mind-muscle
cp9
mattshea
jacksepticeye
situationship
omiyage
hanabi
kyushu
canon-g7x
aeropostale
650d
tectronic
mark-6
retroarch
non-jailbroken
degarmo
rbf
lanab4eva
eight-ish
deju
busan
dscwx300


adt
marrakesh
ctaa
marth
hyperness
aquabogging
amazo
brethrens
chit-chatty
stonie
expand-there
go-ah
super-sarcastic
petsmart
mquan
ddzeets
colley
estée
suvcrossover
giarda
essie
chinchilly
2007-08.
g-girls.jpeg
cheez-its
dreyfuss
50¢
alanah
a-l-a-n-a-h-f-c
alanahfc
gtas
haymitch
mini-bags
slowly-
michonne
bosluna
odx
koreatown
super-attracted
camp-y
those..
lindt
everyb
suesub
fransisco
musicartist
doregama1
ifunny
checkvile
hasbro
starkiller
izombie
sandburg
2k11
phifer
hoursday
the..
g99
jean-martin
fortier
myhalosgood
nosscr
favorite-
dlp
dikes-
c-curl
ganjo
mm-hmm
b17
mega-con
slush-puppies
fluttershy
storm-well
thing-but
askmathison
battlespot
mikinlvoe
m-i-k-i-n-l-v-o-e
sydneyppatterson
toule
kindofdry
tunesia
pro-lactin
youtube-er
skyfly
sum2015
keagan
beason
infernape
sceptile
blaney
home-schooler
stormstrong
cmking03
blondest
detail..
binondo
step-nephew
beauty-related
widdled
mid-trip
duresta
pacuto
jehoshaphat
patrons-only
because..
j.k.
gross-
longas
guin
swim-two-birds
mi

13826

In [13]:
# save words in dataframe, sort and save the file.
keys = [key for key in frequencies]
values = [frequencies[key] for key in keys]
embedding = []
for key in keys:
    if key=='text-start' or key=='text-end' :
        embedding.append(model['-'])
    else:
        embedding.append(model[key])
        
df = pd.DataFrame({'word':keys,'count':values, 'vector':embedding},columns=['word','count','vector'])
newdf=df.sort_values('count',ascending=0).reset_index(drop=1)

#save word to ID map
# PATH = '/USERS/d8182103/'
with open(PATH+'firstimpressionV2/tfrecord/text/word2ID.txt','w') as f:
    for w in newdf['word']:
        f.write(w+'\n')
    # padding word
    f.write('padding-mark'+'\n')
# save word embedding matrix
np.save(PATH+'firstimpressionV2/tfrecord/text/embedding_matrix',np.vstack(newdf['vector']))
# save UNK word vector
np.save(PATH+'firstimpressionV2/tfrecord/text/UNK',np.vstack(model['UNK']).reshape(1,300))

# put SOS and SOE at the beginning

In [19]:
PATH = '/USERS/d8182103/'
embedding_matrix = np.load(PATH+'firstimpressionV2/tfrecord/text/embedding_matrix.npy')

In [20]:
SOS_SOE = np.array(embedding_matrix[7:9], copy=True)
firsttwo = np.array(embedding_matrix[0:2], copy=True)

In [29]:
firsttwo.mean()

0.00026366691

In [30]:
SOS_SOE.mean()

-0.0022383332

In [31]:
embedding_matrix[7:9].mean()

0.00026366691

In [32]:
embedding_matrix[0:2].mean()

-0.0022383332

In [25]:
embedding_matrix[7:9] = firsttwo
embedding_matrix[0:2] = SOS_SOE

In [33]:
np.save(PATH+'firstimpressionV2/tfrecord/text/embedding_matrix',embedding_matrix)

# save text training data

In [19]:
# read labels and text(if it has)
def savetext(setname, dataset='firstimpressionV2', modality='text',filename = 'text_and_labels.csv', PATH = '/USERS/d8182103'):
    csvpath = PATH+'/%s/%s/%s' %(dataset, setname, filename)
    df = pd.read_csv(csvpath)
    print('open csv: %s' %csvpath)
    textpath = PATH+'/%s/tfrecord/%s/%s/%s.txt' %(dataset, modality, setname, setname)
    print('save txt: %s' %textpath)
    fw=open(textpath.split('.txt')[0]+'_videos_without_transcription.txt','w')
    print('save txt: %s' %(textpath.split('.txt')[0]+'_videos_without_transcription.txt'))
    with open(textpath,'w') as f:
        for i in range(len(df)):
            # text pre-processing
            # make sure it has transcription
            if type(df.loc[i,'text']) is not type('s'):
                fw.write(df.loc[i,'video']+'\n')
                print(df.loc[i,'text'],type(df.loc[i,'text']),df.loc[i,'video'])
            else:
                # filter text
                re_text = textfilter(df.loc[i,'text'])
                # delete half-words
                if '-' in re_text[-5:]:
                    splits = re_text.split('-')
                    re_text= '-'.join(splits[:-1])+ ' '+splits[-1]
                if '-' in re_text[:5]:
                    splits = re_text.split('-')
                    re_text=splits[0] + ' ' +'-'.join(splits[1:])
                # tokenize text
                #remove / and ' from word
                tokens = [re.sub(r"[\/|\'|\=]",'',w) for w in word_tokenize(re_text)]
                tokens = ' '.join(tokens)
                # add start and set mark
                tokens = 'text-start '+tokens+' text-end\n'
                f.write(tokens)
    fw.close()

In [20]:
for setname in ['train','test','vali']:
    savetext(setname)

open csv: /USERS/d8182103/firstimpressionV2/train/text_and_labels.csv
save txt: /USERS/d8182103/firstimpressionV2/tfrecord/text/train/train.txt
save txt: /USERS/d8182103/firstimpressionV2/tfrecord/text/train/train_videos_without_transcription.txt
nan <class 'float'> iYVJt41_q7M.002.mp4
nan <class 'float'> 4LZJvOecyM8.005.mp4
nan <class 'float'> YC3X1DcnUrk.000.mp4
nan <class 'float'> ztyBhnjtrz0.000.mp4
nan <class 'float'> JTmq4k4uQCY.003.mp4
nan <class 'float'> KRo-x2uoHUg.003.mp4
nan <class 'float'> HhC2cGFFZeY.000.mp4
nan <class 'float'> cRDYrvxRJ6U.001.mp4
open csv: /USERS/d8182103/firstimpressionV2/test/text_and_labels.csv
save txt: /USERS/d8182103/firstimpressionV2/tfrecord/text/test/test.txt
save txt: /USERS/d8182103/firstimpressionV2/tfrecord/text/test/test_videos_without_transcription.txt
nan <class 'float'> JmAQlC-FEV8.000.mp4
nan <class 'float'> _plk5k7PBEg.004.mp4
nan <class 'float'> L-rmZZP_wj8.005.mp4
open csv: /USERS/d8182103/firstimpressionV2/vali/text_and_labels.csv
sa

# save big5 labels

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
PATH = '/USERS/d8182103/'

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_dataframe(df,dataset,setname):
    # read file containing the video name that will not be used
    delete_list = PATH+('firstimpression%s/tfrecord/text/%s/%s_videos_without_transcription.txt' %(dataset,setname,setname))
    if dataset =='V2':
        df = df.drop(columns=['text'])
    try:
        with open(delete_list,'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print('firstimpression V1')
    else:
        # get indexes that should be removed from datafram
        remove_indexes = []
        for l in lines:
            remove_indexes.append(df[df['video']==l[:-1]].index[0])
        # drop rows of those indexes
        df = df.drop(np.array(remove_indexes))
    return df

In [5]:
def save_big5(dataset,setname):
    # read dataframe
    df = pd.read_csv(PATH+'firstimpression%s/%s/text_and_labels.csv' %(dataset,setname))
    # get correct df
    newdf = get_dataframe(df,dataset, setname)
    # make df for tfrecord writing
    if dataset =='V2':
        data_info = pd.DataFrame({'name':['video','big5','job'],
                                 'type':['string','float32','float32'],
                                 'shape':[(),(5,),()],
                                 'isbyte':[True,False,False],
                                 "length_type":['fixed','fixed','fixed'],
                                 "default":[np.NaN,np.NaN,np.NaN]})
    else:
        data_info = pd.DataFrame({'name':['video','big5'],
                                 'type':['string','float32'],
                                 'shape':[(),(5,)],
                                 'isbyte':[True,False],
                                 "length_type":['fixed','fixed'],
                                 "default":[np.NaN,np.NaN]})

    data_info_path = PATH+'firstimpression%s/tfrecord/big5/data_info.csv' %dataset
    data_info.to_csv(data_info_path,index=False)

    tfr = TFrecorder()
    writer = tf.python_io.TFRecordWriter(PATH+'firstimpression%s/tfrecord/big5/%s/%s.tfrecord' %(dataset,setname,setname))
    num_examples = len(newdf)
    for i in np.arange(num_examples):
        # 要写到tfrecord文件中的字典

        features = {}
        # get information from dataframe
        one_user = newdf.iloc[i]
        # write video name
        video = one_user['video']
        tfr.feature_writer(data_info.iloc[0], video, features)
        #print(video)
        # write big5
        big5 = one_user[['ope', 'con', 'ext', 'agr', 'neu']]
        tfr.feature_writer(data_info.iloc[1], big5, features)
        #print(big5)
        # write job label
        if dataset =='V2':
            job = one_user['job']
            tfr.feature_writer(data_info.iloc[2], job, features)
            #print(job)
        tf_features = tf.train.Features(feature= features)
        tf_example = tf.train.Example(features = tf_features)
        tf_serialized = tf_example.SerializeToString()
        writer.write(tf_serialized)
    writer.close()

In [6]:
for dataset in ['V1','V2']:
    for setname in ['train','vali']:
        save_big5(dataset,setname)
        print(dataset,setname)
save_big5('V2','test')
print('V2','test')

firstimpression V1
V1 train
firstimpression V1
V1 vali
V2 train
V2 vali
V2 test


# model

In [12]:
import sys
import pandas as pd
import numpy as np
from fastText import load_model
from tfrecorder import TFrecorder
import tensorflow as tf
PATH = '/USERS/d8182103/'
# 实例化该类
tfr = TFrecorder()
dataset = 'firstimpressionV2/'
setname = 'vali'
modalname = 'text'
# 当前写的样本数
num_examples_per_file = 1000
num_so_far = 0
PATH = '/USERS/d8182103/'
datapath='%s/%s/tfrecord/%s/%s/' %(PATH,dataset,modalname, setname)
data_info_path = datapath+'data_info.csv'

In [3]:
import sys
import pandas as pd
import numpy as np
from fastText import load_model
from tfrecorder import TFrecorder
import tensorflow as tf
PATH = '/USERS/d8182103/'
tfr = TFrecorder()

In [30]:
def input_fn_maker(dataset ='V2', setname='train', batchsize =2, shuffle_buffer = 0, epoch =1):
    def input_fn():
        # big five label dataset
        # path
        label_info_path = '/USERS/d8182103/firstimpressionV1/tfrecord/big5/data_info.csv'
        label_dataset_path = '/USERS/d8182103/firstimpression%s/tfrecord/big5/%s/%s.tfrecord' %(dataset,setname,setname)
        # dataset
        tfr.get_dataset(paths=[label_dataset_path], data_info=label_info_path)
        label_dataset = tfr.dataset_raw

        # text dataset
        # path
        text_dataset_path = '/USERS/d8182103//firstimpression%s/tfrecord/text/%s/%s.txt' %(dataset,setname,setname)
        # dataset
        text_dataset = tf.data.TextLineDataset(text_dataset_path)
        text_dataset = text_dataset.map(lambda x: tf.string_split([x], " ").values)


        # merge
        merge_datasets_list = (label_dataset, text_dataset)
        merge_dataset = tf.data.Dataset.zip(merge_datasets_list)

        # dataset transformation
        # shuffle
        if shuffle_buffer>10:
            merge_dataset = merge_dataset.shuffle(buffer_size=shuffle_buffer)
        # batch_padding
        padding_info = ({'big5':[None]}, [None])
        pad_word = tf.constant('padding-mark')
        padding_value = ({'big5':0.0}, pad_word)

        merge_dataset = merge_dataset.padded_batch(batch_size, padded_shapes = padding_info, padding_values = padding_value)
        merge_dataset = merge_dataset.repeat(epoch)
        iterator = merge_dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

In [31]:
input_fn()

read dataframe from /USERS/d8182103/firstimpressionV2/tfrecord/big5/train/train.tfrecord x 1
   default  isbyte length_type  name shape     type
0      NaN   False       fixed  big5   [5]  float32


({'big5': <tf.Tensor 'IteratorGetNext_6:0' shape=(?, ?) dtype=float32>},
 <tf.Tensor 'IteratorGetNext_6:1' shape=(?, ?) dtype=string>)

In [27]:
merge_dataset = merge_dataset.make_one_shot_iterator().get_next()

In [12]:
merge_dataset

({'big5': <tf.Tensor 'IteratorGetNext_1:0' shape=(?, ?) dtype=float32>},
 <tf.Tensor 'IteratorGetNext_1:1' shape=(?, ?) dtype=string>)

In [50]:
merge_dataset = tf.data.Dataset.zip((label_dataset,text_dataset))
padding_info = ({'big5':[None],'job':[]},[None])
padding_value = ({'big5':0.0,'job':0.0},tf.constant('padding-mark'))
pad_dataset = merge_dataset.padded_batch(2, padded_shapes = padding_info,padding_values=padding_value)
#.make_one_shot_iterator().get_next()

In [52]:
sample_merge=pad_dataset.make_one_shot_iterator().get_next()

In [17]:
sess.run([sample_label['big5'] , sample_label['job'], sample_text])

TypeError: tuple indices must be integers or slices, not str

In [28]:
sess.run([merge_dataset[0]['big5'] , merge_dataset[1]])

[array([[ 0.6111111 ,  0.64077669,  0.41121495,  0.56043959,  0.625     ],
        [ 0.3888889 ,  0.14563107,  0.35514018,  0.46153846,  0.25      ]], dtype=float32),
 array([[b'text-start', b'maintenance', b',', b'so', b'making', b'sure',
         b'that', b'you', b're', b'not', b'letting', b'in', b'drown', b'in',
         b'the', b'pan', b',', b'that', b'you', b're', b'cleaning', b'it',
         b'our', b'regularly', b',', b'that', b'you', b're', b'using',
         b'the', b'pan', b'regularly', b'.', b'those', b'things', b'are',
         b'all', b'great', b',', b'but', b'they', b're', b'all', b'a',
         b'little', b'higher', b'maintenance', b'that', b'typical',
         b'fountain', b'techniques', b'.', b'an', b'ounce', b'of',
         b'prevention', b'is', b'worth', b'a', b'pound', b'of', b'cure',
         b'.', b'text-end'],
        [b'text-start', b'job', b'and', b'depend', b'on', b'you', b'two',
         b'for', b'your', b'income', b'when', b'it', b'gets', b'to', b'the',
    

In [56]:
sess.run([sample_merge[0]['big5'] , sample_merge[0]['job'], sample_merge[1]])

[array([[ 0.54444444,  0.44660193,  0.55140185,  0.49450549,  0.46875   ],
        [ 0.62222224,  0.80582523,  0.48598132,  0.78021979,  0.72916669]], dtype=float32),
 array([ 0.48598132,  0.62616825], dtype=float32),
 array([[b'text-start', b'but', b'i', b'ca', b'nt', b'say', b'i', b've',
         b'kept', b'count', b'of', b'how', b'many', b'times', b'i', b'put',
         b'my', b'finger', b'in', b'the', b'oven', b'.', b'sorry', b'.',
         b'what', b's', b'the', b'color', b'of', b'the', b'ocean', b'?',
         b'the', b'ocean', b'is', b'actually', b'bright', b'orange', b',',
         b'and', b'if', b'you', b'see', b'it', b',', b'it', b's', b'blue',
         b',', b'you', b're', b'actually', b'...', b'text-end'],
        [b'text-start', b'because', b'as', b'again', b',', b'inaudible',
         b'for', b'scottish', b'business', b',', b'that', b's', b'not',
         b'healthy', b'in', b'the', b'current', b'environment', b'and',
         b'it', b's', b'certainly', b'not', b'going', b

In [16]:
sess=tf.InteractiveSession()

In [26]:
text_dataset = tf.data.TextLineDataset([datapath+'vali.txt'])
text_dataset = text_dataset.map(lambda x: tf.string_split([x], " ").values)
padding_info = [None]
padding_value = tf.constant('padding-mark')
dataset2=dataset2.padded_batch(4, padded_shapes = padding_info,padding_values=padding_value)

In [2]:
def input_fn():
    dataset = tf.data.TextLineDataset([datapath+'train.txt'])
    dataset2 = dataset.map(lambda x: tf.string_split([x], " ").values)
    padding_info = [None]
    padding_value = tf.constant('padding-mark')
    dataset2=dataset2.padded_batch(4, padded_shapes = padding_info,padding_values=padding_value)
    return dataset2.make_one_shot_iterator().get_next()

In [3]:
EMBEDDING_SIZE=300
def model_fn(features, mode):
    feature_map = features
    table = tf.contrib.lookup.index_table_from_file(vocabulary_file=PATH+'firstimpressionV2/tfrecord/text/word2ID.txt', num_oov_buckets=1)
    ids = table.lookup(features)
    
    
    embedding_matrix = tf.Variable(np.load(PATH+'firstimpressionV2/tfrecord/text/embedding_matrix.npy'),trainable=0)
    padding_word = tf.Variable(tf.zeros([1,EMBEDDING_SIZE]),trainable=0)
    UNK_vector = tf.Variable(np.load(PATH+'firstimpressionV2/tfrecord/text/UNK.npy'),trainable=0)
    
    embeddings = tf.concat(values=[embedding_matrix,padding_word,UNK_vector], axis=0)
    embed = tf.nn.embedding_lookup(embeddings, ids)
    logits = tf.reduce_mean(input_tensor = tf.reduce_mean(input_tensor = embed,axis=-1),axis=-1)
    h1 = tf.Variable(tf.constant(0.1, shape=[1]))
    loss = tf.losses.mean_squared_error(labels=logits, predictions=logits*h1)
    predictions = {
        'word':feature_map,
        "ids": ids,
        "embed":embed
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)


    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1)
        train_op = optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss)

In [4]:
tester = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir="ok")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ok', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd953df7400>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
rm -rf ok

In [6]:
tester.train(input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ok/model.ckpt.
INFO:tensorflow:loss = 5.16466e-06, step = 1
INFO:tensorflow:global_step/sec: 275.946
INFO:tensorflow:loss = 2.18662e-09, step = 101 (0.364 sec)
INFO:tensorflow:global_step/sec: 335.949
INFO:tensorflow:loss = 3.48568e-13, step = 201 (0.297 sec)
INFO:tensorflow:global_step/sec: 325.651
INFO:tensorflow:loss = 2.29785e-14, step = 301 (0.307 sec)
INFO:tensorflow:global_step/sec: 359.084
INFO:tensorflow:loss = 2.37005e-18, step = 401 (0.278 sec)
INFO:tensorflow:global_step/sec: 359.017
INFO:tensorflow:loss = 0.0, step = 501 (0.279 sec)
INFO:tensorflow:global_step/sec: 341.058
INFO:tensorflow:loss = 0.0, step = 601 (0.293 sec)
INFO:tensorflow:global_step/sec: 339.545
INFO:tensorflow:loss = 0.0, step = 701 (0.294 sec)
INFO:tensorflow:global_step/sec: 339.358
INFO:tensorflow:loss = 0.0, step = 801 (0.295 sec)
INFO:tensorflow:global_step/sec: 334.795
INFO:tensorflow:loss = 0.0, step = 901 (0

In [7]:
OUT= list(tester.predict(input_fn))

INFO:tensorflow:Restoring parameters from ok/model.ckpt-1498


In [8]:
a0 = OUT[80]

In [9]:
a0

{'embed': array([[-0.0092, -0.0478, -0.038 , ...,  0.0041, -0.0209, -0.0178],
        [-0.0069, -0.0343, -0.0589, ...,  0.2033,  0.0737, -0.0768],
        [ 0.0156,  0.0752, -0.078 , ...,  0.0882, -0.0882, -0.0096],
        ..., 
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]], dtype=float32),
 'ids': array([    7,    15,    18,     9,   102,   145,  1412,   183,    10,
            0,     2,    49,     3,    94,     1,    56,   317,    56,
           10,   343,     0,    10,   251,    15,     6,   588,    13,
         1258,    10,     2,    99,   294,    18,  1412,   183,    10,
            0,    28,    10,    12,    36,     2,   153,    58,     1,
            8, 13826, 13826, 13826, 13826, 13826, 13826, 13826, 13826, 13826]),
 'word': array([b'text-start', b'...', b'is', b'a', b'little', b'bit', b'higher',
        b'than', b'tha

In [11]:
model['...']-a0['embed'][1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0